# DataFrame

 Usando Pandas, vai ler cada CSV e combinando-os em um único DataFrame.

1078 musicas


DataFrame combinado: (628446, 9) 628446 linhas e 9 colunas

Cada linha nesse DataFrame representa uma mensagem MIDI individual, capturando um evento específico dentro da música, e não uma música inteira

Formato do DataFrame merged_df: (628446, 10)

# Colunas e Tipos de Dados:


*   **track** (int): Indica a trilha MIDI à qual a mensagem pertence, sendo um número inteiro (0 para a primeira trilha, 1 para a segunda, etc.).

*   **time** (float): Representa o tempo acumulado em segundos desde o início da música até o evento da mensagem MIDI, um valor de ponto flutuante que permite precisão temporal.

*   **type** (object): Descreve o tipo de mensagem MIDI, como 'note_on' (nota tocada), 'note_off' (nota liberada), 'set_tempo' (mudança de andamento), e 'end_of_track' (fim da trilha), sendo um tipo de dado de objeto (string).

*   **channel** (float): O canal MIDI (0-15) ao qual a mensagem se aplica. Valores NaN (Not a Number) podem aparecer para tipos de mensagem que não utilizam um canal (como set_tempo ou end_of_track).

*   **note** (float): O número da nota MIDI (0-127), relevante para mensagens 'note_on' e 'note_off'. Para outros tipos de mensagens, o valor é NaN.

*   **velocity** (float): A intensidade ou volume com que a nota é tocada (0-127), aplicado a mensagens 'note_on' e 'note_off'. Também pode ser NaN para mensagens não relacionadas a notas.

*   **value** (float): Um valor genérico usado por outros tipos de mensagens MIDI, como control_change ou pitchwheel. Geralmente é NaN para mensagens de nota.

*   **tempo** (float): O tempo da música em microssegundos por batida, relevante apenas para mensagens 'set_tempo'. Para a maioria das outras mensagens, será NaN.

*   **music_name** (object): O nome original do arquivo MIDI (sem a extensão .csv), que foi adicionado durante o carregamento para identificar a qual música cada mensagem pertence. ***Esta coluna é crucial para associar os dados da música aos rótulos emocionais.***












# O QUE FOI FEITO


*   Combinação dos CSVs: Todos os arquivos CSV individuais (resultantes da conversão MIDI) são lidos e concatenados em um único e grande DataFrame chamado combined_df. Este DataFrame representa cada mensagem MIDI individual como uma linha, contendo informações como tipo de evento, tempo, nota, velocidade, etc., e o nome da música à qual pertence (music_name)

*   Carregamento e Pré-processamento dos Rótulos Emocionais: O arquivo label.csv fornecido pelo dataset EMOPIA é carregado. As colunas são inspecionadas e, crucialmente, a coluna ID é renomeada para music_name. Além disso, a categoria emocional (Q1, Q2, Q3 ou Q4) é extraída do music_name e salva em uma nova coluna emotion_quadrant, que servirá como o rótulo (variável alvo)


*   Merge dos Dados MIDI com os Rótulos: O combined_df (contendo todas as mensagens MIDI) é mesclado com o DataFrame de rótulos (emopia_labels_unique) utilizando a coluna music_name como chave. Isso resulta no merged_df, onde cada mensagem MIDI agora carrega também o rótulo emocional de sua música. Isso é fundamental, pois permite associar as características de áudio (ou MIDI) com a emoção.


*   engenharia de features: Criaçao de novas features, como densidade, proporção de eventos e variaçao ritmica. Agrupa o merged_df por música e calcula diversas características estatísticas (como total de eventos, duração, pitch médio, velocidade, etc.) para cada música, consolidando os dados em um features_df onde cada linha representa uma música com suas features e o rótulo de emoção. É *importante* notar que, diferentemente dos DataFrames anteriores (combined_df e merged_df que têm uma linha por evento MIDI), no *features_df* cada linha realmente representa uma única música, contendo suas features agregadas e o rótulo de emoção correspondente.

* Escalonamento/ Normalizaçao das features



**features_df é salvo no Drive, sendo acessível e reutilizável em futuros Colab para as etapas de treinamento e avaliação de modelos de IA**

**MyDrive/INF 420 - IA/TRABALHO FINAL/dados_processados/emopia_features_with_labels.pkl'**

In [ ]:
import pandas as pd
import os
from google.colab import drive
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
try:
    drive.mount('/content/drive')
    print("Google Drive montado com sucesso.")
except Exception as e:
    print(f"Erro ao montar o Google Drive (pode já estar montado): {e}")

Erro ao montar o Google Drive (pode já estar montado): Error: credential propagation was unsuccessful


In [ ]:
# pasta onde os CSVs foram salvos
#csv_output_folder = '/content/drive/MyDrive/INF 420 - IA/TRABALHO FINAL/musicas/CSV'
csv_output_folder = '/content/drive/MyDrive/INF 420 - IA/TRABALHO FINAL/MUSICAS/EMOPIA/CSV'

In [ ]:
# lista para armazenar cada DataFrame lido
all_dataframes = []

In [ ]:
# iterar sobre os arquivos na pasta e carregar cada CSV
print(f"Buscando arquivos CSV na pasta: {csv_output_folder}")
if os.path.exists(csv_output_folder):
    for filename in os.listdir(csv_output_folder):
        if filename.lower().endswith('.csv'):
            file_path = os.path.join(csv_output_folder, filename)
            try:
                df = pd.read_csv(file_path)

                music_name = os.path.splitext(filename)[0]
                df['music_name'] = music_name

                all_dataframes.append(df)
                print(f"Carregado: {filename}")
            except Exception as e:
                print(f"Erro ao carregar '{filename}': {e}")

    if all_dataframes:
        combined_df = pd.concat(all_dataframes, ignore_index=True)
        print("\nTodos os CSVs foram combinados em um único DataFrame.")
        print(f"Formato do DataFrame combinado: {combined_df.shape}")
        #print("Primeiras 5 linhas do DataFrame combinado:")
        #print(combined_df.head())
        #print("\nInformações sobre as colunas e tipos de dados:")
        #combined_df.info()

        # combined_df contem os dados de todas as suas musicas MIDI em um formato tabular pronto para a engenharia de features
    else:
        print("Nenhum arquivo CSV encontrado na pasta especificada ou erro ao carregar.")
else:
    print(f"A pasta '{csv_output_folder}' não foi encontrada. Verifique o caminho ou se os CSVs foram gerados nela.")

Buscando arquivos CSV na pasta: /content/drive/MyDrive/INF 420 - IA/TRABALHO FINAL/MUSICAS/EMOPIA/CSV
A pasta '/content/drive/MyDrive/INF 420 - IA/TRABALHO FINAL/MUSICAS/EMOPIA/CSV' não foi encontrada. Verifique o caminho ou se os CSVs foram gerados nela.


In [ ]:
combined_df.shape

NameError: name 'combined_df' is not defined

In [ ]:
combined_df.tail(5)

In [ ]:
combined_df.head(5)

In [ ]:
#ver quantas musicas tem
print(combined_df['music_name'].unique())
print(f"Total de músicas únicas carregadas: {len(combined_df['music_name'].unique())}")

# USANDO label.csv

* Drive: acessa os arquivos do dataset EMOPIA
* carrega o arquivo label.csv, que contém os rótulos emocionais associados a cada música do dataset
* processa esse arquivo de rótulos, coluna de id da música --> music_name e extraindo a categoria emocional (Q1, Q2, Q3, Q4) diretamente do nome de cada arquivo
* fusão desses rótulos emocionais com o combined_df, contém todas as mensagens MIDI de cada música

* Agora, cada mensagem MIDI ta diretamente associada ao quadrante emocional da música

In [ ]:
try:
    drive.mount('/content/drive')
except Exception as e:
    print(f"Drive already mounted or error: {e}")


emopia_labels_path = '/content/drive/MyDrive/INF 420 - IA/TRABALHO FINAL/EMOPIA_1.0/EMOPIA_1.0/label.csv'

if os.path.exists(emopia_labels_path):
    emopia_labels_df = pd.read_csv(emopia_labels_path)
    print("DataFrame de rótulos EMOPIA (label.csv) carregado com sucesso.")
    print("Primeiras 5 linhas do DataFrame de rótulos:")
    print(emopia_labels_df.head())
    print(f"Colunas do DataFrame de rótulos: {emopia_labels_df.columns.tolist()}")
else:
    print(f"ERRO: Arquivo de rótulos EMOPIA (label.csv) NÃO FOI ENCONTRADO em: {emopia_labels_path}")
    exit()

# --- PREPARAÇÃO PARA O MERGE ---

# renomear a coluna 'ID' para 'music_name'
# rodando pela segunda vez: ja trocou nome

#emopia_labels_df = emopia_labels_df.rename(columns={'ID': 'music_name'})

# extrair emoção (Q1, Q2, Q3, Q4) da coluna 'music_name'
emopia_labels_df['emotion_quadrant'] = emopia_labels_df['music_name'].str.split('_').str[0]


emopia_labels_unique = emopia_labels_df[['music_name', 'emotion_quadrant']].drop_duplicates()

print("\nDataFrame de rótulos preparado para merge:")
print(emopia_labels_unique.head())
print(f"Número de músicas únicas no DataFrame de rótulos: {len(emopia_labels_unique)}")

merged_df = pd.merge(combined_df, emopia_labels_unique, on='music_name', how='left')


print("\nDataFrame combinado com rótulos emocionais (merged_df):")
print(f"Formato do DataFrame merged_df: {merged_df.shape}")
print("Primeiras 10 linhas do DataFrame merged_df (com rótulos):")
print(merged_df.head(10))

# verifica se todas as músicas encontraram um rótulo
nan_labels = merged_df['emotion_quadrant'].isnull().sum()
if nan_labels > 0:
    print(f"\nATENÇÃO: {nan_labels} linhas no merged_df têm rótulos ausentes (NaN) na coluna 'emotion_quadrant'.")
    print("Isso pode indicar que algumas músicas no seu combined_df não foram encontradas no label.csv.")
else:
    print("\nTodos os arquivos MIDI no combined_df foram rotulados com sucesso!")

print("\nContagem de músicas por quadrante de emoção:")
print(merged_df['emotion_quadrant'].value_counts())

# ENGENHARIA DE FEATURES

salvar para um novo colab o dataFrame features_df

ADICIONANDO NOVAS FEATURES

- note_density (Densidade de Notas)
  - total_midi_events / duration
- note_on_proportion (Proporção de Eventos Note_On)
  - num_note_on_events / total_midi_events
- std_time_diff_between_notes (Variação Rítmica)
  - Calcula a diferença entre os times de eventos note_on consecutivos usando diff() e retorna o desvio padrão dessas diferenças de tempo

AGRUPANDO CADA LINHA POR UMA MUSICA

 - a estratégia de primeiro decompor a música em seus eventos MIDI individuais e depois agregá-los novamente em um features_df permite extrair uma riqueza de informações descritivas da música que seria impossível obter com uma representação mais simplificada desde o início. Isso aumenta a qualidade e a relevância das features para o modelo de IA, melhorando as chances de aprender e prever corretamente a emoção associada à música.

In [ ]:
# agrupar por music_name e extrair features
features_df = merged_df.groupby('music_name').agg(
    total_midi_events=('type', 'count'),
    duration=('time', lambda x: x.max() if x.max() > 0 else 0),
    num_unique_tracks=('track', 'nunique'),
    num_unique_channels=('channel', 'nunique'),
    avg_note_pitch=('note', 'mean'),
    std_note_pitch=('note', 'std'),
    min_note_pitch=('note', 'min'),
    max_note_pitch=('note', 'max'),
    pitch_range=('note', lambda x: x.max() - x.min() if pd.notna(x.max()) and pd.notna(x.min()) else 0), # Melhorar o tratamento de NaNs para pitch_range
    avg_note_velocity=('velocity', 'mean'),
    std_note_velocity=('velocity', 'std'),
    num_note_on_events=('type', lambda x: (x == 'note_on').sum()),
    num_note_off_events=('type', lambda x: (x == 'note_off').sum()),
    num_set_tempo_events=('type', lambda x: (x == 'set_tempo').sum()),
    emotion_quadrant=('emotion_quadrant', 'first')
).reset_index()

# ----------------- NOVAS FEATURES -------------------

# Densidade de notas: total_midi_events / duration
# garantir que 'duration' != zero
features_df['note_density'] = features_df.apply(
    lambda row: row['total_midi_events'] / row['duration'] if row['duration'] > 0 else 0,
    axis=1
)


# Proporção de eventos note_on: num_note_on_events / total_midi_events
# 'total_midi_events' não seja zero
features_df['note_on_proportion'] = features_df.apply(
    lambda row: row['num_note_on_events'] / row['total_midi_events'] if row['total_midi_events'] > 0 else 0,
    axis=1
)


# Variação rítmica: std_time_diff_between_notes
# passo adicional: precisa do 'time' dos eventos 'note_on' para cada musica

def calculate_rhythmic_std(group):
    # Filtra apenas eventos 'note_on'
    note_on_times = group[group['type'] == 'note_on']['time']
    if len(note_on_times) > 1:
        time_diffs = note_on_times.diff().dropna()
        if len(time_diffs) > 0:
            return time_diffs.std()
    return 0


rhythmic_std = merged_df.groupby('music_name').apply(calculate_rhythmic_std).reset_index(name='std_time_diff_between_notes')


features_df = pd.merge(features_df, rhythmic_std, on='music_name', how='left')


features_df['std_time_diff_between_notes'] = features_df['std_time_diff_between_notes'].fillna(0)




# NaNs --> 0 para features numéricas, exceto para 'emotion_quadrant'
# 'pitch_range' e 'std_note_pitch' podem ser NaN se houver poucas notas.
for col in ['avg_note_pitch', 'std_note_pitch', 'min_note_pitch', 'max_note_pitch', 'pitch_range',
            'avg_note_velocity', 'std_note_velocity', 'note_density', 'note_on_proportion', 'std_time_diff_between_notes']:
    if col in features_df.columns:
        features_df[col] = features_df[col].fillna(0)


print("\nDataFrame de Features por Música (features_df):")
print(f"Formato do DataFrame features_df: {features_df.shape}")
print("Primeiras 5 linhas do DataFrame features_df:")
print(features_df.head())

print("\nContagem de Músicas por Quadrante no features_df:")
print(features_df['emotion_quadrant'].value_counts())
print(f"Total de músicas únicas no features_df: {len(features_df)}")


In [ ]:
features_df.shape

# Escalonamento/Normalização das Features

- garante que as características (features) com diferentes escalas e intervalos de valores contribuam de forma equitativa para o treinamento dos modelos.  

  - Muitos algoritmos são sensíveis à escala das features; sem esse processamento, características com valores maiores poderiam dominar o processo de aprendizado, distorcendo os resultados e prejudicando a performance do modelo.
  - Ao aplicar o StandardScaler, as features são transformadas para ter uma média de zero e um desvio padrão de um, padronizando-as e otimizando o desempenho dos algoritmos subsequentes.

In [ ]:
numeric_features = features_df.select_dtypes(include=['number']).columns.tolist()

scaler = StandardScaler()
features_df[numeric_features] = scaler.fit_transform(features_df[numeric_features])


print("\nDataFrame de Features Escaladas (features_df):")
print(features_df.head())

# salvar no drive

In [ ]:
output_directory = '/content/drive/MyDrive/INF 420 - IA/TRABALHO FINAL/dados_processados/'
output_filename = 'emopia_features_with_labels.pkl' # 2 opçoes: .pkl ou .parquet
output_path = os.path.join(output_directory, output_filename)

os.makedirs(output_directory, exist_ok=True)

features_df.to_pickle(output_path)  # .pkl
# Ou: features_df.to_parquet(output_path, index=False) # Para .parquet (requer `pip install pyarrow`)

print(f"\nDataFrame de features salvo com sucesso em: {output_path}")

In [ ]:
output_directory = '/content/drive/MyDrive/INF 420 - IA/TRABALHO FINAL/dados_processados/'
output_filename_csv = 'emopia_features_with_labels.csv'
output_path_csv = os.path.join(output_directory, output_filename_csv)
features_df.to_csv(output_path_csv, index=False)
print(f"DataFrame de features salvo em CSV: {output_path_csv}")